<a href="https://colab.research.google.com/github/chechae/24w_audio/blob/main/KoBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import glob
from collections import defaultdict
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from tokenizers import Tokenizer
from transformers import pipeline
import warnings
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Ko-Bart

사투리 -> 표준어 번역 프로세싱

In [ ]:
dir = '/content/drive/MyDrive/kobart/Training/label1' #경상도 텍스트 데이터

print(dir)

files = glob.glob(f"{dir}/*.json")
df = pd.DataFrame(index=range(2), columns=["사투리","표준어"])

for file in files:
    # print(file)
    _dict = defaultdict(list)

    with open(file) as f:
        json_data = json.load(f)
        # JSON 파일의 utterance 키 값 아래에 리스트형태로 데이터가 존재했다.
        data = json_data.get("utterance")

        for item in data:
            # standard_form(표준어)와 dialect_form(사투리)가 일치하지 않는다면?
            standard = item.get("standard_form")
            dialect = item.get("dialect_form")
            if standard != dialect:
                _dict["표준어"].append(standard)
                _dict["사투리"].append(dialect)

    # 한 JSON 파일에서 데이터 추출이 끝나면 비어있던 데이터프레임에 추가
    append_df = pd.DataFrame(_dict)
    df = pd.concat([df, append_df])

df.to_csv("data.tsv",sep="\t")

/content/drive/MyDrive/kobart/Training/label1


In [ ]:
len(df)

336482

In [ ]:
df.head()

,사투리,표준어
27,지가 지 기도하고 뭐~ 예배하고 했는데,자기가 자기 기도하고 뭐~ 예배하고 했는데
28,그~ 능인고 그쪽 아이가?,그~ 능인고 그쪽 아니니?
29,대박이노.,대박이네.
30,대박이네 혹시 그~ 고등학교 친구들 중에 지금 스님된 사람 없나?,대박이네 혹시 그~ 고등학교 친구들 중에 지금 스님된 사람 없니?
31,아~ 공부 쫌 했지.,아~ 공부 조금 했지.


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('사투리').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('표준어').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['표준어'].value_counts()
    for x_label, grp in _df_2.groupby('사투리')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('사투리')
_ = plt.ylabel('표준어')

## 전처리

In [ ]:
import re

step1 = re.compile(r'&\w+&')
step2 = re.compile(r'\(*\)')
step3 = re.compile(r'\{[^}]*\}')
step4 = re.compile(r'\-[^}]*\-')

#예시 문장
text1 = "&name2& 님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?"
text2 = "어~ {반가움} &name123&! (()) 반갑고 ㅋㅋ -오- 오랜만이다"

# 무식해보이지만 일단 확인해보자!
for text in [text1,text2]:
    after_step1=step1.sub('',text)
    after_step2=step2.sub('',after_step1)
    after_step3=step3.sub('',after_step2)
    after_step4=step4.sub('',after_step3)

    print(f"Text : {text}")
    print(f"Step 1 : {after_step1}")
    print(f"Step 2 : {after_step2}")
    print(f"Step 3 : {after_step3}")
    print(f"Step 4 : {after_step4}")
    print("\n\n")

Text : &name2& 님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 1 :  님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 2 :  님은 뭐~ {laughing}  힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 3 :  님은 뭐~   힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 4 :  님은 뭐~   힘들 때  위로가 되어 준 노래 같은 게 있으신가요?



Text : 어~ {반가움} &name123&! (()) 반갑고 ㅋㅋ -오- 오랜만이다
Step 1 : 어~ {반가움} ! (()) 반갑고 ㅋㅋ -오- 오랜만이다
Step 2 : 어~ {반가움} !  반갑고 ㅋㅋ -오- 오랜만이다
Step 3 : 어~  !  반갑고 ㅋㅋ -오- 오랜만이다
Step 4 : 어~  !  반갑고 ㅋㅋ  오랜만이다





## 데이터 정제

In [ ]:
df = pd.read_csv(f'data.tsv',sep='\t')

# data.tsv를 불러왔더니 nan 값이 들어있어 일단 삭제해줬다.
df = df.dropna()

# 아까 지정해뒀던 정규표현식들
step1 = re.compile(r'&\w+&')
step2 = re.compile(r'\(*\)')
step3 = re.compile(r'\{[^}]*\}')
step4 = re.compile(r'\-[^}]*\-')


def make_cleaned_text(text):
    after_step1=step1.sub('',text)
    after_step2=step2.sub('',after_step1)
    after_step3=step3.sub('',after_step2)
    after_step4=step4.sub('',after_step3)

    return after_step4

for idx, row in df.iterrows():
    df.at[idx, '사투리'] = make_cleaned_text(row['사투리'])
    df.at[idx, '표준어'] = make_cleaned_text(row['표준어'])

df = df.iloc[:, 1:]

df.to_csv('data_fixed.tsv', index=False, sep='\t')

In [ ]:
df.tail()

,사투리,표준어
336477,지가 지 기도하고 뭐~ 예배하고 했는데,자기가 자기 기도하고 뭐~ 예배하고 했는데
336478,그~ 능인고 그쪽 아이가?,그~ 능인고 그쪽 아니니?
336479,대박이노.,대박이네.
336480,대박이네 혹시 그~ 고등학교 친구들 중에 지금 스님된 사람 없나?,대박이네 혹시 그~ 고등학교 친구들 중에 지금 스님된 사람 없니?
336481,아~ 공부 쫌 했지.,아~ 공부 조금 했지.


## 모델 파라미터 설정

In [ ]:
os.makedirs(name='saved_model', mode=0o777, exist_ok=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

# 모델 학습 후에 가중치 저장할 폴더
model_path= r"/content/drive/MyDrive/kobart/Training/model"
# 모델 초기 가중치 로드할 곳
model_name = "gogamza/kobart-base-v2"
# 데이터셋 파일
data_root='/content/'

# 내가 학습한 모델 가중치의 유무에 따라 분기 처리
if os.path.exists(f'{model_path}/pytorch_model.bin'):
    print("Use Customized Model")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    print("Use Pretrained Model")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Training Arguments

args = {
    'num_train_epochs': 20,
    'per_device_train_batch_size': 32,
    'per_device_eval_batch_size': 32,
    'overwrite_output_dir': True,
    'eval_steps': 5,
    'save_steps': 5,
    'warmup_steps': 5,
    'evaluation_strategy': "steps",
    'prediction_loss_only': True,
    'save_total_limit': 3
}

Use Pretrained Model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
text1=row[0]
text2=row[1]
print(text1);print(text2)

31
아~ 공부 쫌 했지.


In [ ]:
from torch.utils.data import Dataset

class TextStyleTransferDataset(Dataset):
    def __init__(self, df,tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row=self.df.iloc[index]
        text1=row[0] #표준어
        text2=row[1] #사투리
        target_style_name = '표준어'

        # Tokenizer를 허깅페이스의 레포지토리에서 가져왔기 때문에
        # 테스트 코드를 참고해서 모델 입력 형식을 만들었다.
        encoder_text = f"{target_style_name} 말투로 변환:{text1}"
        decoder_text = f"{text2}{self.tokenizer.eos_token}"
        model_inputs = self.tokenizer(encoder_text, max_length=64, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            labels = tokenizer(decoder_text, max_length=64, truncation=True)
        model_inputs['labels'] = labels['input_ids']
        del model_inputs['token_type_ids']

        return model_inputs

# 만들어뒀던 데이터셋을 불러와서 데이터프레임을 만들어준다
def make_df(data_root):
    df = pd.read_csv(f'{data_root}/data_fixed.tsv',sep='\t')
    # 주로 쓰이는 방법 같지는 않지만,, Train Data와 Test Data를 8:2 비율로 나눠준다
    rate=int(len(df)*0.1)
    df_train,df_test = df[rate:],df[:rate]

    print(f'Train DataFrame length : {len(df_train)},Test DataFrame length : {len(df_test)}')
    return df_train,df_test

def make_dataset(df):
    df_train,df_test = df

    train_dataset = TextStyleTransferDataset(df_train,tokenizer)
    test_dataset = TextStyleTransferDataset(df_test,tokenizer)

    return train_dataset,test_dataset

In [ ]:
df = make_df(data_root)

Train DataFrame length : 302832,Test DataFrame length : 33648


## 학습

In [ ]:
!pip install accelerate -U

In [ ]:
pip install --upgrade accelerate

In [ ]:
pip show accelerate

Name: accelerate
Version: 0.28.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
# from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,DataCollatorForSeq2Seq

from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

warnings.filterwarnings("ignore")

df = make_df(data_root)
train_dataset,test_dataset=make_dataset(df)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)


training_args = Seq2SeqTrainingArguments(
    **args,
    output_dir=model_path,
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


Train DataFrame length : 302832,Test DataFrame length : 33648


In [ ]:
print(trainer)

In [ ]:
# 모델 학습 진행
try:
    trainer.train()
except Exception as e:
    print(f"Failed to train model caused by {e}")


Step,Training Loss,Validation Loss
5,No log,1.868206
10,No log,1.295924
15,No log,0.882511
20,No log,0.739255
25,No log,0.598314


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

Step,Training Loss,Validation Loss
5,No log,1.868206
10,No log,1.295924
15,No log,0.882511
20,No log,0.739255
25,No log,0.598314


KeyboardInterrupt: 

In [ ]:
try:
    trainer.save_model(model_path)
    print("Model saved successfully.")
except Exception as e:
    print(f"Failed to save model caused by {e}")

## Test (Validation)

AI hub에 있는 텍스트 데이터에는 테스트용(라벨 없는거)이 따로 없긴 하다.
그래서 이전에 그냥 trainset에서 8:2로 split시켜서 validation용으로 퉁치긴 했지만
일단 나중에 오디오 -> 텍스트(STT)실행하고 나서 어차피 데이터는 바꿔서 껴야되기 때문에 그때 validation도 상황에 맞게 데이터 변경해야함


In [ ]:
from transformers import pipeline

nlg_pipeline=pipeline('text2text-generation',model=model_path,tokenizer=model_name)

In [ ]:
def generate_text(pipe, text, num_return_sequences, max_length):
    target_style_name = "표준어"
    text = f"{target_style_name} 말투로 변환:{text}"
    out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
    #num_return_sequences의 값에 따라서 반환되는 텍스트의 개수가 바뀐다. 만약 3으로 지정했다면 길이가 3인 리스트에 담겨서 값이 반환될 것임!
    return [x['generated_text'] for x in out]


print("Write 'q' to exit")
while True:
    src_text=input("Dialect to translate(입력받을 사투리) : ")
    if src_text == 'q':
        break
    target_text_ko=generate_text(nlg_pipeline,src_text,num_return_sequences=1,max_length=64)[0]
    print(f"Translated Standard (표준어로 출력): {target_text_ko}")